In [1]:
# Import necessary packages
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Read sample orders file, containing all past purchases for customers
orders = pd.read_csv(r"C:\Users\santo\OneDrive\Documents\Career\Training\Python\Sample_Data\RFM\sales_data.csv",sep=',')

In [3]:
orders.head()

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Price,Unit Cost
0,SO - 000101,In-Store,WARE-UHY1004,12/31/2017,5/31/2018,6/14/2018,6/19/2018,USD,6,15,259,12,5,0.075,1963.1,1001.181
1,SO - 000102,Online,WARE-NMK1003,12/31/2017,5/31/2018,6/22/2018,7/2/2018,USD,14,20,196,27,3,0.075,3939.6,3348.660
2,SO - 000103,Distributor,WARE-UHY1004,12/31/2017,5/31/2018,6/21/2018,7/1/2018,USD,21,16,213,16,1,0.050,1775.5,781.220
3,SO - 000104,Wholesale,WARE-NMK1003,12/31/2017,5/31/2018,6/2/2018,6/7/2018,USD,28,48,107,23,8,0.075,2324.9,1464.687
4,SO - 000105,Distributor,WARE-NMK1003,4/10/2018,5/31/2018,6/16/2018,6/26/2018,USD,22,49,111,26,8,0.100,1822.4,1476.144


In [4]:
#Convert Order Quantity to float data type
orders['Order Quantity'] = orders['Order Quantity'].astype(float)

In [5]:
#Update column names
orders = orders.rename(columns={"OrderDate": "order_date", 
                   "_CustomerID": "customer",
                   "OrderNumber": "order_id",
                    "Order Quantity": "order_quantity",
                      "Unit Price": "unit_price"})

In [6]:
#Add column for order_total
orders['grand_total'] = orders.order_quantity * orders.unit_price
#orders['grand_total'] = orders.apply(lambda row: (row['Unit Price']*['Order Quantity']), axis=1)

# Create the RFM Table

In [7]:
# Make the date_placed column datetime
orders['order_date'] = pd.to_datetime(orders['order_date'])

In [8]:
#Identify last order date to use as reference point for 'recency' - in this case is 12th December 2020
orders['order_date'].max()

Timestamp('2020-12-30 00:00:00')

In [9]:
import datetime as dt
NOW = dt.datetime(2020,12,30)

In [10]:
orders.head()

,order_id,Sales Channel,WarehouseCode,ProcuredDate,order_date,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,customer,_StoreID,_ProductID,order_quantity,Discount Applied,unit_price,Unit Cost,grand_total
0,SO - 000101,In-Store,WARE-UHY1004,12/31/2017,2018-05-31,6/14/2018,6/19/2018,USD,6,15,259,12,5.0,0.075,1963.1,1001.181,9815.5
1,SO - 000102,Online,WARE-NMK1003,12/31/2017,2018-05-31,6/22/2018,7/2/2018,USD,14,20,196,27,3.0,0.075,3939.6,3348.660,11818.8
2,SO - 000103,Distributor,WARE-UHY1004,12/31/2017,2018-05-31,6/21/2018,7/1/2018,USD,21,16,213,16,1.0,0.050,1775.5,781.220,1775.5
3,SO - 000104,Wholesale,WARE-NMK1003,12/31/2017,2018-05-31,6/2/2018,6/7/2018,USD,28,48,107,23,8.0,0.075,2324.9,1464.687,18599.2
4,SO - 000105,Distributor,WARE-NMK1003,4/10/2018,2018-05-31,6/16/2018,6/26/2018,USD,22,49,111,26,8.0,0.100,1822.4,1476.144,14579.2


In [16]:
rfmTable = orders.groupby('customer').agg({'order_date': lambda x: (NOW - x.max()).days, # Recency - no of days between last order on dataset and last order for this customer
                                        'order_id': lambda x: len(x),      # Frequency - number of orders
                                        'grand_total': lambda x: x.sum()}) # Monetary Value - total value of orders


In [17]:
rfmTable['order_date'] = rfmTable['order_date'].astype(int)


In [18]:
rfmTable.rename(columns={'order_date': 'recency', 
                         'order_id': 'frequency', 
                         'grand_total': 'monetary_value'}, inplace=True)

# Validating the RFM Table

In [19]:
rfmTable.head()

,recency,frequency,monetary_value
customer,,,
1,7,152,1322278.5
2,7,135,1346264.5
3,8,181,1831947.5
4,3,167,1770582.2
5,28,159,1609232.8


# Determining RFM Quartiles

In [20]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
#Send quantiles to dictionary for easier use
quantiles = quantiles.to_dict()

In [21]:
quantiles

{'recency': {0.25: 2.0, 0.5: 3.0, 0.75: 7.0},
 'frequency': {0.25: 151.25, 0.5: 157.5, 0.75: 167.75},
 'monetary_value': {0.25: 1515516.55,
  0.5: 1621580.9000000001,
  0.75: 1816336.5}}

# Creating the RFM segmentation table

In [22]:
rfmSegmentation = rfmTable

In [23]:
# We create two classes for the RFM segmentation since, being high number of days since last purchase is bad, while high frequency and monetary value is good.

In [24]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [25]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [26]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [27]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
1,7,152,1322278.5,3,3,4,334
2,7,135,1346264.5,3,4,4,344
3,8,181,1831947.5,4,1,1,411
4,3,167,1770582.2,2,2,2,222
5,28,159,1609232.8,4,2,3,423


# Find the most valuable customers 

In [29]:
rfmSegmentation[rfmSegmentation['RFMClass']=='111'].head(10)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
12,1,210,2248332.4,1,1,1,111
13,2,171,1868321.8,1,1,1,111
29,0,179,2112221.9,1,1,1,111
32,0,173,1856355.6,1,1,1,111
39,2,176,1823941.0,1,1,1,111
